In [65]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os


import cv2
import shutil
import numpy as np

from PIL import Image, ImageDraw
from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir, create_yolo_annotation_from_mask

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# validate model
model = YOLO('runs/segment/augmented/weights/best.pt')
val_results = model.val(data='val.yaml', task='detect')

Ultralytics YOLOv8.1.0 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (AMD Ryzen 9 5900X 12-Core Processor)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented... 44963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 44963/44963 [02:47<00:00, 268.33it/s]


val: New cache created: D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2811/2811 [47:40<00:00,  1.02s/it]


                   all      44963     295094      0.977       0.85      0.888      0.815      0.975      0.844      0.882       0.74
Speed: 0.6ms preprocess, 41.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\val


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BD59166E90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [17]:
model = YOLO('runs/segment/augmented/weights/best.pt')
inference_results = model(source='datasets/CF_simulation/real_image/test/', save=True, project='real_image_test' ,name='real_image_test')


image 1/4 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test\hdt5_0010.tif: 384x480 9 ink traces, 35.0ms
image 2/4 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test\hdt5_1482.tif: 384x480 6 ink traces, 35.0ms
image 3/4 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test\hdt5_1483.tif: 384x480 8 ink traces, 35.0ms
image 4/4 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test\hdt_0146.tif: 384x480 2 ink traces, 36.0ms
Speed: 1.0ms preprocess, 35.3ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 480)
Results saved to real_image_test\real_image_test


In [93]:
import time
#img = cv2.imread('datasets/CF_simulation/real_image/test/hdt5_0010.tif')
img = Image.open('real_image_test/real_image_test/hdt5_1482.tif') 
draw = ImageDraw.Draw(img) 
w, h = img.size
for coos in inference_results[1].masks.xy:
    start = time.time()
    # get LMS estimation
    x, y = coos[:,0:1], coos[:,1:]
    X = np.concatenate((np.ones_like(x), x), axis=-1)
    XT = X.transpose()
    XTX = XT @ X
    beta = np.linalg.inv(XTX) @ (XT @ y)
    
    beta=beta.reshape(-1)
    intercept, slope = beta[0], beta[1]
    #print(intercept, slope)
   
    # get projection distances
    a = np.array([[1], [slope]])
    B = np.concatenate((x, y), axis=-1).transpose()
    B -= intercept
    P = a@(a.transpose()@B)/(a.transpose()@a)
    E = B-P
    distances = (E**2).sum(axis=0)**.5
    estimated_widths = distances.mean()*2
    print('Elapsed time: {:.1e}'.format(time.time()-start))
    draw_points = [(0, int(intercept)), (w, int(intercept+slope*w))]
    draw.line(draw_points, fill='green', width=10)
    print(estimated_widths)


img.show()


Elapsed time: 1.0e-03
100.77009705560855
Elapsed time: 0.0e+00
165.89550848533736
Elapsed time: 0.0e+00
233.85270834443244
Elapsed time: 1.0e-03
250.75363201220335
Elapsed time: 0.0e+00
194.96343984759795
Elapsed time: 0.0e+00
293.83791321244706


In [16]:
import torch
torch.cuda.empty_cache()